In [ ]:
!pip install selenium
!pip install pandas 
!pip install time
!pip install sklearn

In [48]:
from selenium import webdriver
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from selenium.webdriver.common.keys import Keys

In [2]:
#Please make sure to enter the gmail and password for GitHub in the text file
f = open("./credentials.txt", "r")
email = f.readline()
password = f.readline()

In [3]:
driver = webdriver.Chrome('C:/chromedriver')
driver.maximize_window()
driver.get('https://github.com/login')

<ipython-input-3-60b9e604fc8c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chromedriver')


In [4]:
df = pd.DataFrame(columns=['Date','User','Repository'])

In [5]:
#Logging in GitHub using the credentials provided in the text file
driver.find_element_by_xpath('//*[@id="login_field"]').send_keys(email)
driver.find_element_by_xpath('//*[@id="password"]').send_keys(password)
driver.find_element_by_xpath('//*[@id="login"]/div[4]/form/div/input[11]').click()

<ipython-input-5-849c549a1816>:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="login_field"]').send_keys(email)
<ipython-input-5-849c549a1816>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="password"]').send_keys(password)
<ipython-input-5-849c549a1816>:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="login"]/div[4]/form/div/input[11]').click()


In [6]:
#Gets the date of each commit
def GetDate(l):
    d = l[0].replace(",", "").split(" ")
    date = d[-3] + " " + d[-2] + " " + d[-1]
    return date

In [7]:
#Gets the users on each repository commits section
def GetUsers(l):
    users = []
    for i in range(len(l)):
        word = l[i].split(" ")
        if (len(word) >= 4):
            if (word[1] == 'committed'):
                users.append(word[0])
            elif (word[1] == 'and' and word[3] == 'committed'):
                    users.append(word[0])
                    users.append(word[2])
    return users


In [8]:
#This function will scrape the repository links. More keywords can be added to the keys list below.
def GetRepoLinks():
    keys = ['Python', 'C++', 'Ruby', 'Java']
    links = []
    for i in keys:
        driver.get('https://github.com/')
        driver.find_element_by_xpath('/html/body/div[1]/header/div[3]/div/div/form/label/input[1]').send_keys(i)
        driver.find_element_by_xpath('/html/body/div[1]/header/div[3]/div/div/form/label/input[1]').send_keys(Keys.ENTER)
        c = 0
        for j in driver.find_elements_by_class_name('v-align-middle'):
            if (j.get_attribute('href') != None):
                links.append(j.get_attribute('href'))
                c = c + 1

                if (c == 8):
                    break
    return links

In [9]:
links = GetRepoLinks()

<ipython-input-8-653cb6fa9aea>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/header/div[3]/div/div/form/label/input[1]').send_keys(i)
<ipython-input-8-653cb6fa9aea>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/header/div[3]/div/div/form/label/input[1]').send_keys(Keys.ENTER)
<ipython-input-8-653cb6fa9aea>:9: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  for j in driver.find_elements_by_class_name('v-align-middle'):


In [10]:
def ScrollDown():
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [11]:
#GetName returns the name of the repository the scraper is on

def GetName():
    l = driver.find_element_by_xpath('//*[@id="repository-container-header"]/div[1]/div/div').text.split("\n")
    name = l[0] + l[1] + l[2]
    return name

In [12]:
#Mining all the user commits for each repository

for link in links:
    link = link + '/commits/master'
    driver.get(link)
    cond = False
    url2 = ''
    cond = False
    for j in range(30):
        if (cond == True):
            break
        #print ("Url is", link)
        url = driver.current_url

        name = GetName()
        time.sleep(3.5)

        l = driver.find_elements_by_class_name('TimelineItem-body')
        text = []
        for i in l:
            text.append(i.text)
        for i in text:
            l = driver.find_elements_by_class_name('TimelineItem-body')
            i = i.split("\n")
            date = GetDate(i)
            if (date.split(" ")[-1] == '2018'):
                cond = True
                break

            users = GetUsers(i)
            for user in users:
                row1 = pd.Series([date, user, name], index=df.columns)
                df = df.append(row1,ignore_index=True)


                ScrollDown()
                time.sleep(0.5)

        try:
            ScrollDown()
            #finding old pages of that repository to be scraped
            driver.find_element_by_link_text("Older").click()
            #print ("Clicked on", link)
            time.sleep(0.5)
        except:
            pass
        url2 = driver.current_url


    

<ipython-input-11-4df7130935b0>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  l = driver.find_element_by_xpath('//*[@id="repository-container-header"]/div[1]/div/div').text.split("\n")
<ipython-input-12-9a93285f98b3>:16: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  l = driver.find_elements_by_class_name('TimelineItem-body')
<ipython-input-12-9a93285f98b3>:21: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  l = driver.find_elements_by_class_name('TimelineItem-body')
<ipython-input-12-9a93285f98b3>:39: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("Older").click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="repository-container-header"]/div[1]/div/div"}
  (Session info: chrome=107.0.5304.88)
Stacktrace:
Backtrace:
	Ordinal0 [0x005B1ED3+2236115]
	Ordinal0 [0x005492F1+1807089]
	Ordinal0 [0x004566FD+812797]
	Ordinal0 [0x004855DF+1005023]
	Ordinal0 [0x004857CB+1005515]
	Ordinal0 [0x004B7632+1209906]
	Ordinal0 [0x004A1AD4+1120980]
	Ordinal0 [0x004B59E2+1202658]
	Ordinal0 [0x004A18A6+1120422]
	Ordinal0 [0x0047A73D+960317]
	Ordinal0 [0x0047B71F+964383]
	GetHandleVerifier [0x0085E7E2+2743074]
	GetHandleVerifier [0x008508D4+2685972]
	GetHandleVerifier [0x00642BAA+532202]
	GetHandleVerifier [0x00641990+527568]
	Ordinal0 [0x0055080C+1837068]
	Ordinal0 [0x00554CD8+1854680]
	Ordinal0 [0x00554DC5+1854917]
	Ordinal0 [0x0055ED64+1895780]
	BaseThreadInitThunk [0x752EFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77267BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77267B8E+238]


In [38]:
#Creating the final dataset

final_df = pd.DataFrame(columns=['Repository','Effort'])

In [16]:
df.to_csv('file2.csv', header=False, index=False)


In [44]:
#Creating a new dataframe that shows the effort for each repository, based on the current dataset. 

for i in df['Repository'].unique():
    df2 = df[df['Repository'] == i]
    num_of_commits = len(df2)
    num_of_active_users = len(df2['User'].unique())
    num_of_active_days = len(df2['Date'].unique())
    effort = num_of_commits * num_of_active_users * num_of_active_days
    print (effort)
    row1 = pd.Series([i, effort], index=final_df.columns)
    final_df = final_df.append(row1,ignore_index=True)
    

3046042
56700
4032
30
88200
157500
103950
111720


In [52]:
final_df

,Repository,Effort
0,TheAlgorithms/Python,3046042
1,geekcomputers/Python,56700
2,walter201230/Python,4032
3,injetlee/Python,30
4,kubernetes-client/python,88200
5,Show-Me-the-Code/python,157500
6,xxg1413/python,103950
7,grpc/grpc,111720


In [54]:
#Min Max Scaling the Effort column

column = 'Effort'
final_df[column] = (final_df[column] - final_df[column].min()) / (final_df[column].max() - final_df[column].min())    


In [55]:
final_df

,Repository,Effort
0,TheAlgorithms/Python,1.0
1,geekcomputers/Python,0.018605
2,walter201230/Python,0.001314
3,injetlee/Python,0.0
4,kubernetes-client/python,0.028946
5,Show-Me-the-Code/python,0.051697
6,xxg1413/python,0.034117
7,grpc/grpc,0.036668
